In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('BTCUSDT_DataSet.csv')
df = data.copy()

###### Keeping Buy and Sell price in different columns

In [3]:
df['bidPrice'] = np.where(df['isBuyerMaker'] == False, df['Price'], 0)
df['askPrice'] = np.where(df['isBuyerMaker'] == True, df['Price'], 0)

###### Initializing all the columns needed for strategy implementation

In [14]:
test = df.copy()
test['MyBid'] = 0
test['MyAsk'] = 0
test['MyTradeSize'] = 0
test['Trades'] = np.nan
test['TradedQty'] = 0
test['TradedQty(USD)'] = 0
test['Inventory'] = 0
test['MyExecutedBid'] = 0
test['MyExecutedAsk'] = 0

###### Explanations
1. MyBid : to store my current Bid Price
2. MyAsk : to store my current Ask Price
3. MyTradeSize : 1000 USD
4. Trades : to update BUY/SELL
5. TradedQty : taking minimum of {Qty(available in tick data), MyTradeSize}
6. TradedQty(USD) : to convert Traded Quantity to USD
7. Inventory : to keep track of my current position (-5000 <= Inventory <= 5000)
8. MyExecutedBid : My last Bid which got executed at current row/tick (for PnL calculation)
9. MyExecutedAsk : My last Ask which got executed at current row/tick (for PnL calculation)

###### Market Order Code Explained
1. Used to exiting positions in last 20 ticks
2. If my InventorySize > 0, the I'll place continuous SELL Orders untill the inventory Size becomes zero
3. Else if my InventorySize < 0, the I'll place continuous BUY Orders untill the inventory Size becomes zero
4. Insize while loop, I'm checking if my current tick if Buy/Sell order, thenaccording to my requirement I'm updating all the      values

###### Market Order Implementation to exit positions (assuming we can exit all our positions in last 20 ticks keping tradeSize = InventorySize)

In [9]:
def marketOrder(test, l):
    if(test['Inventory'].iloc[l] > 0):     #place continuous sell orders        
        #print('Inventory Size :', test['Inventory'].iloc[l])
        
        while((l < len(test)-1) and test['Inventory'].iloc[l] != 0):
            if(test['isBuyerMaker'].iloc[l] == False):
                test['Trades'].iloc[l] = 'SELL'
                test['MyExecutedAsk'].iloc[l] = test['Price'].iloc[l] #because its a market order
                test['MyTradeSize'].iloc[l] = test['Inventory'].iloc[l] / test['Price'].iloc[l]
                #test['TradedQty'].iloc[l] = test[['Qty', 'MyTradeSize']].iloc[l].min()
                
                test['TradedQty'].iloc[l] = test['MyTradeSize'].iloc[l]
                test['TradedQty(USD)'].iloc[l] = (-1)*test['Price'].iloc[l]*test['TradedQty'].iloc[l]            
                test['Inventory'] = test['TradedQty(USD)'].cumsum()

                #print('Inventory Size :', test['Inventory'].iloc[l])
                if(test['Inventory'].iloc[l] == 0):
                    break
            l = l+1
        
    elif(test['Inventory'].iloc[l] < 0):     #place continuous buy orders
        #print('Inventory Size :', test['Inventory'].iloc[l])
        
        while((test['Inventory'].iloc[l] != 0) and (l < len(test)-1)):
            if(test['isBuyerMaker'].iloc[l] == True):
                test['Trades'].iloc[l] = 'BUY'
                test['MyExecutedBid'].iloc[l] = test['Price'].iloc[l] #because its a market order                
                test['MyTradeSize'].iloc[l] = (-1)*test['Inventory'].iloc[l] / test['Price'].iloc[l]
                #test['TradedQty'].iloc[l] = test[['Qty', 'MyTradeSize']].iloc[l].min()                
                
                test['TradedQty'].iloc[l] = test['MyTradeSize'].iloc[l]
                test['TradedQty(USD)'].iloc[l] = test['Price'].iloc[l]*test['TradedQty'].iloc[l]            
                test['Inventory'] = test['TradedQty(USD)'].cumsum()

                #print('Inventory Size :', test['Inventory'].iloc[l])
                if(test['Inventory'].iloc[l] == 0):
                    break
            l = l+1
    

###### Market Making Strategy Code Explained
1. Using data set excluding last 20 ticks, we enter the while loop
2. MyBid price = MidPrice - 3basisPoints, here MidPrice is assumed as Trade Price
3. MyAsk Price = MidPrice + 3basisPoints, here MidPrice is assumed as Trade Price
4. currentTime : tracking currentTime to take care of 100ms of latency time
5. If currentTime+200 ms value exists in the 'time' column, then we assign it to j, k and l,
   implying we wait for 200ms after calculating MyBid and MyAsk price
6. After waiting for 200ms, we start comparing My Ask/Bid Price to the trade prices in upcoming ticks to find a trade match

###### Strategy Implementation (updating Limit orders )

In [15]:
%%time
i=0
size = len(test) - 20
while(i < size):
    
    bsp3 = .0003   #calculating 3basispoints
    test['MyBid'].iloc[i] = test['Price'].iloc[i] - bsp3 * test['Price'].iloc[i] 
    test['MyAsk'].iloc[i] = test['Price'].iloc[i] + bsp3 * test['Price'].iloc[i]    
    
    currentTime = test['time'].iloc[i]
    
    if(len(test[test['time'] > currentTime+200].index) > 0):
        j = test[test['time'] > currentTime+200].index[0]
        k = j        
# below variable will be used when we reach the end of limit orders, and where we need to exit our positions using market order
        l = j
        
    else:
        break
    
    # For SELL trade
    if(test['Inventory'].iloc[j-1] >= -4000):
        
        # Skipping ticks untill my trade conditions is not fulfilled
        while((j < size) & (test['bidPrice'].iloc[j] < test['MyAsk'].iloc[i])):
            if(j+1 < size):
                j = j+1
            else:
                break
        
        # Updating SELL trade if the trade conditions met     
        if((j < size)&(test['bidPrice'].iloc[j] >= test['MyAsk'].iloc[i])):
            test['Trades'].iloc[j] = 'SELL'
            test['MyExecutedAsk'].iloc[j] = test['MyAsk'].iloc[i]            
            test['MyTradeSize'].iloc[j] = 1000 / test['Price'].iloc[j]
            #test['TradedQty'].iloc[j] = test[['Qty', 'MyTradeSize']].iloc[j].min()
            
            test['TradedQty'].iloc[j] = test['MyTradeSize'].iloc[j]
            test['TradedQty(USD)'].iloc[j] = (-1)*test['Price'].iloc[j]*test['TradedQty'].iloc[j]            
            test['Inventory'] = test['TradedQty(USD)'].cumsum()
    
    # For BUY trade
    if(test['Inventory'].iloc[k-1] <= 4000):
        
        while((k<size) & ((test['askPrice'].iloc[k] == 0)|(test['askPrice'].iloc[k] > test['MyBid'].iloc[i]))):
            if(k+1 < size):
                k = k+1
            else:
                break
        
        #Updating BUY trade if the trade conditions met
        if((k < size)&(test['isBuyerMaker'].iloc[k] == True)&(test['askPrice'].iloc[k] <= test['MyBid'].iloc[i])):
            test['Trades'].iloc[k] = 'BUY'
            test['MyExecutedBid'].iloc[k] = test['MyBid'].iloc[i]
            test['MyTradeSize'].iloc[k] = 1000 / test['Price'].iloc[k]
            #test['TradedQty'].iloc[k] = test[['Qty', 'MyTradeSize']].iloc[k].min()
            
            test['TradedQty'].iloc[k] = test['MyTradeSize'].iloc[k]
            test['TradedQty(USD)'].iloc[k] = test['Price'].iloc[k]*test['TradedQty'].iloc[k]
            test['Inventory'] = test['TradedQty(USD)'].cumsum()
    
# If both of my last Bid/Ask price got executed, then we update the nextIndex to max(j,k) to keep track of latest inventory size
    if(test['Trades'].iloc[j] == 'SELL' and test['Trades'].iloc[k] == 'BUY'):
        nextIndex = max(j, k)
        
# If one of my last Bid/Ask price got executed, then we update the nextIndex to min(j,k) to keep track of latest inventory size       
    elif((test['Trades'].iloc[j] == 'SELL' and test['Trades'].iloc[k] != 'BUY') or 
         (test['Trades'].iloc[k] == 'BUY' and test['Trades'].iloc[j] != 'SELL')):
        nextIndex = min(j, k) 
        
# If none of my last Bid/Ask price got executed, then we recalculate the Bid/Ask Price     
    elif(test['Trades'].iloc[k] != 'BUY' and test['Trades'].iloc[j] != 'SELL'):
        nextIndex = l
#         marketOrder(test, l)     
#         break
    
    # To keep track of my last trade tick, so I can control the Inventory size not to go beyond 5000  
    currentTime = test['time'].iloc[nextIndex] 
    
    # I'll wait for 200ms, then I will calculate MyBid/MyAsk price for next trade in next iteration
    if(len(test[test['time'] > currentTime+200].index) > 0):
        i = test[test['time'] > currentTime+200].index[0] 
    else:
        break

#Call the marketOrder function at the end to execute market orders to make sure inventory becomes empty 
marketOrder(test, len(test)-20)

Wall time: 11min 37s


###### Number of buy/sell trades

In [16]:
len(test[(test['Trades'] == 'BUY')|(test['Trades'] == 'SELL')])

452

###### Filtering the trades data

In [17]:
test[(test['Trades'] == 'BUY')|(test['Trades'] == 'SELL')].round(8)

,Index,Trade_Id,Price,Qty,QuoteQty,time,isBuyerMaker,isBestMatch,bidPrice,askPrice,MyBid,MyAsk,MyTradeSize,Trades,TradedQty,TradedQty(USD),Inventory,MyExecutedBid,MyExecutedAsk
88,88,1274557632,43173.65,0.09036,3901.171014,1646092801205,False,True,43173.65,0.00,0.0,0.0,0.023162,SELL,0.023162,-1000.0,-1000.0,0.000000,43172.958003
170475,170475,1274728019,43147.06,0.00613,264.491478,1646100350736,True,True,0.00,43147.06,0.0,0.0,0.023177,BUY,0.023177,1000.0,0.0,43147.061997,0.000000
170589,170589,1274728133,43154.36,0.09038,3900.291057,1646100351211,False,True,43154.36,0.00,0.0,0.0,0.023173,SELL,0.023173,-1000.0,-1000.0,0.000000,43154.122354
171432,171432,1274728976,43128.00,0.05407,2331.930960,1646100385116,True,True,0.00,43128.00,0.0,0.0,0.023187,BUY,0.023187,1000.0,0.0,43128.237646,0.000000
171522,171522,1274729066,43144.56,0.06164,2659.430678,1646100390063,False,True,43144.56,0.00,0.0,0.0,0.023178,SELL,0.023178,-1000.0,-1000.0,0.000000,43142.598898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808875,4808875,1279366419,42451.03,0.01927,818.031348,1646351902691,False,True,42451.03,0.00,0.0,0.0,0.023557,SELL,0.023557,-1000.0,4000.0,0.000000,42450.221247
4808915,4808915,1279366459,42466.91,0.17686,7510.697703,1646351905707,False,True,42466.91,0.00,0.0,0.0,0.023548,SELL,0.023548,-1000.0,3000.0,0.000000,42466.736200
4809151,4809151,1279366695,42454.49,0.00940,399.072206,1646351951269,True,True,0.00,42454.49,0.0,0.0,0.023555,BUY,0.023555,1000.0,4000.0,42454.729759,0.000000
4809307,4809307,1279366851,42454.01,0.00006,2.547241,1646351999124,False,True,42454.01,0.00,0.0,0.0,0.094220,SELL,0.094220,-4000.0,0.0,0.000000,42454.010000


###### Exporting the Trades data to a csv file

In [18]:
test[(test['Trades'] == 'BUY')|(test['Trades'] == 'SELL')].round(8).to_csv('TradesFinal.csv', index=False)